# Chapter 2: Being Clear and Direct

**Scenario:** You built an AI assistant, but it talks too much. It gives long introductions and excuses instead of just answering. You need to teach it to be **concise** and **opinionated**.

In this notebook, you will learn:
- ✂️ **Direct Answers**: How to stop the model from saying "Here is the answer..."
- 😤 **Forcing Decisions**: Getting an opinion from a neutral model.
- 🎯 **The Golden Rule**: Be specific to get specific results.

---

## 1. Setup

Let's get our tool ready again.

In [1]:
# Install tools
!pip install -q litellm python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [3]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

## 2. Cutting the Extra Words

AI models are very polite. If you ask for a list, they often say "Sure! Here is the list you asked for:"...

This is annoying if you are building an app. Let's see it happen:

In [4]:
print(get_completion("List 3 fruits."))

Okay, here are 3 fruits:

1.  Apple
2.  Banana
3.  Strawberry



### The Fix: Be Explicit

Tell the model exactly what **NOT** to do.

In [5]:
prompt = "List 3 fruits. Do not write any introduction. Just give the list."
print(get_completion(prompt, temperature=0.7, max_tokens=1024))

*   Apple
*   Banana
*   Orange



## 3. Forcing a Decision

AI models are trained to be neutral. They don't like to pick sides.

If you ask "Which is better: Summer or Winter?", it will usually say "Both have good points..."

In [6]:
print(get_completion("Which is better: Summer or Winter?"))

Ah, the age-old question! There's no single "better" season, as it truly depends on individual preferences and what you're looking for in a season. Here's a breakdown to help you decide which might be better *for you*:

**Summer:**

*   **Pros:**
    *   **Warm Weather:** Great for outdoor activities like swimming, hiking, picnics, and enjoying the sunshine.
    *   **Longer Days:** More daylight hours mean more time to enjoy activities.
    *   **Vacation Time:** Often associated with school breaks and family vacations.
    *   **Abundant Produce:** Fresh fruits and vegetables are in season.
    *   **Socializing:** People tend to be more active and social during the summer.
    *   **Easier Travel:** Generally, travel is less affected by weather conditions.

*   **Cons:**
    *   **Heat:** Can be uncomfortable or even dangerous in extreme heat.
    *   **Crowds:** Popular tourist destinations can be very crowded.
    *   **Higher Energy Bills:** Air conditioning can be expensive.
   

But sometimes, you **need** a decision. You can force the AI to choose by giving it a specific "role" or "instruction" to pick one.

In [7]:
prompt = """
Which is better: Summer or Winter? 
You must choose exactly one option. Do not be neutral.
"""
print(get_completion(prompt, temperature=0.7, max_tokens=1024))

Summer.



## 4. Exercises

Now you try.

### Exercise 1: One Word Only

**Goal:** Get the model to answer "Paris" and **ONLY** "Paris". No periods, no "The capital is...".


In [8]:
prompt = "What is the capital of France? [Add your instructions here]"

response = get_completion(prompt, temperature=0.7, max_tokens=1024)
print(f"Response: '{response}'")

# Check if specific strict constraints are met
if response == "Paris":
    print("✅ Success!")
else:
    print("❌ Keep trying. Make sure there is no extra whitespace or punctuation.")

Response: 'The capital of France is Paris.
'
❌ Keep trying. Make sure there is no extra whitespace or punctuation.


### Exercise 2: Email Extractor

**Goal:** Extract the email address from the text below. The output should be **just the email**.

Text: "Please contact support@example.com for further assistance."

In [9]:
text = "Please contact support@example.com for further assistance."

prompt = f"""
[Your instructions here]
Text: {text}
"""

response = get_completion(prompt, temperature=0.7, max_tokens=1024)
print(f"Response: '{response}'")

if response.strip() == "support@example.com":
        print("✅ Success!")
else:
    print("❌ Try again.")

Response: 'Okay, I understand. My instruction is missing, so I will provide a few possible interpretations and actions based on what I think you might want:

**Possible Interpretations and Actions:**

**1. Instruction: Extract the email address from the text.**

*   **Action:**  The email address is: `support@example.com`

**2. Instruction:  Rewrite the text to be more friendly/formal/etc.**

*   **Action (More Friendly):** "Need more help? Just shoot us an email at support@example.com!"
*   **Action (More Formal):** "For further inquiries, please direct your correspondence to support@example.com."

**3. Instruction: Check if the email address is valid (format).**

*   **Action:** The email address `support@example.com` appears to be in a valid format.  I cannot guarantee it *works*, but the syntax is correct.

**4. Instruction:  Create a button or link for a website using this text.**

*   **Action:**  I would create a button or link with the text "Contact Support" and the URL would b

## Summary

1.  **Cut the fluff**: Tell the AI to skip introductions.
2.  **Force decisions**: Tell the AI it must choose.
3.  **Be precise**: If you want one word, ask for one word.